## Imports

In [1]:
using LinearAlgebra
using Plots
using GaussQuadrature
using SparseArrays

## Base

In [2]:
function ϕ₁(ξ::Float64)
    return (1.0 - ξ)/2.0
end

function ϕ₂(ξ::Float64)
    return (1.0 + ξ)/2.0
end

function dϕ₁(ξ::Float64)
    return -1.0/2.0
end

function dϕ₂(ξ::Float64)
    return 1.0/2.0
end

dϕ₂ (generic function with 1 method)

## Mapeamento

In [3]:
function map_index_to_coord(index::Int64, ne::Int64)::Float64
    return (index-1)/ne
end

function map_ξ_to_x(ξ::Float64, x_i_inf::Float64, h::Float64)
    return x_i_inf + ((h*(ξ+1))/2.0)
end

map_ξ_to_x (generic function with 1 method)

## DoF Handlers

In [4]:
function monta_LG(ne::Int64)::Matrix{Int64}
    return transpose(hcat(1:ne, 2:ne+1))
end

function monta_EQ(ne::Int64)::Tuple{Int64, Vector{Int64}}
    m = ne-1
    return (m, vcat(m+1, 1:m, m+1))
end

monta_EQ (generic function with 1 method)

## Quadraturas

In [10]:
function quadrature_without_map(func1::Function, func2::Function, P::Vector{Float64}, W::Vector{Float64})::Float64
    quad = 0.0
    
    for (ξ, W) in zip(P, W)
         quad += W * (func1(ξ) * func2(ξ)) 
    end
    
    return quad
end

function quadrature_with_map(func1::Function, func2::Function, e::Int64, ne::Int64, P::Vector{Float64}, W::Vector{Float64})::Float64
    h = 1.0/ne
    x_i_inf = map_index_to_coord(e, ne)
    
    quad = 0.0
    
    for (ξ, W) in zip(P, W)
         quad += W*(func1(map_ξ_to_x(ξ, x_i_inf, h)) * func2(ξ)) 
    end
    
    return quad
end

function quadrature_non_linearity(func1::Function, func2::Function, e::Int64, c̄::Vector{Float64}, EQoLG::Matrix{Float64}, P::Vector{Float64}, W::Vector{Float64})::Float64
    
    quad = 0.0
    
    for (ξ, W) in zip(P, W)
         quad += W * (func1(c̄[EQoLG[1, e]] * ϕ₁(ξ) + c̄[EQoLG[2, e]] * ϕ₂(ξ))) * func2(ξ) 
    end
    
    return quad
end

quadrature_non_linearity (generic function with 1 method)

## Montagem local

In [12]:
function monta_F_local!(Fᵉ::Vector{Float64}, f::Function, e::Int64, ne::Int64, h::Float64, P::Vector{Float64}, W::Vector{Float64})::Vector{Float64}
    
    Fᵉ[1] = h/2 * quadrature_with_map(f, ϕ₁, e, ne, P, W)
    Fᵉ[2] = h/2 * quadrature_with_map(f, ϕ₂, e, ne, P, W)

    return
end

function monta_K_local!(Kᵉ::Matrix{Float64}, α::Float64, β::Float64, h::Float64, P::Vector{Float64}, W::Vector{Float64})::Matrix{Float64}
    
    Kᵉ[1, 1] = (2*α/h) * quadrature_without_map(dϕ₁, dϕ₁, P, W) + (β*h/2) * quadrature_without_map(ϕ₁, ϕ₁, P, W)
    Kᵉ[1, 2] = (2*α/h) * quadrature_without_map(dϕ₁, dϕ₂, P, W) + (β*h/2) * quadrature_without_map(ϕ₁, ϕ₂, P, W)
    Kᵉ[2, 1] = K_local[1, 2]
    Kᵉ[2, 2] = (2*α/h) * quadrature_without_map(dϕ₂, dϕ₂, P, W) + (β*h/2) * quadrature_without_map(ϕ₂, ϕ₂, P, W)    
    
    return
end

function monta_G_local!(Gᵉ::Vector{Float64}, g::Function, e::Int64, c̄::Vector{Float64}, EQoLG::Matrix{Float64}, P::Vector{Float64}, W::Vector{Float64})::Vector{Float64}
    
    Gᵉ[1] = quadrature_non_linearity(g, dϕ₁, e, c̄, EQoLG, P, W)
    Gᵉ[2] = quadrature_non_linearity(g, dϕ₂, e, c̄, EQoLG, P, W)
    
    return
    
end

monta_G_local! (generic function with 2 methods)

## Montagem global

In [14]:
function monta_F_global(f::Function, m::Int64, ne::Int64, npg::Int64, EQoLG::Matrix{Float64})
    h = 1.0/ne
    
    F_global = zeros(m+1)
    F_local = zeros(2)
    P, W = legendre(npg)

    for e in 1:ne
        monta_F_local!(F_local, f, e, ne, h, P, W)
            
        for i in 1:2
            index = EQoLG[i, e]
            F_global[index] += F_local[i]
        end
    end
    
    return F_global[1:m]
end

function monta_K_global(α::Float64, β::Float64, ne::Int64, npg::Int64, EQoLG::Matrix{Float64})
    h = 1.0/ne
    
    K_global = spzeros(m+1,m+1)
    K_local = zeros(2, 2)
    P, W = legendre(npg)
    
    monta_K_local!(K_local, α, β, h, P, W)
    
    for e in 1:ne
        for i in 1:2
            for j in 1:2
                i_global = EQoLG[i, e]
                j_global = EQoLG[j, e]
                
                K_global[i_global, j_global] += K_local[i, j]
            end
        end
    end    
    
    return K_global[1:m, 1:m]
    
end

function monta_G_global(g::Function, c̄::Vector{Float64}, ne::Int64, m::Int64, npg::Int64, EQoLG::Matrix{Float64})
    
    G_global = zeros(m+1)
    G_local = zeros(2)
    P, W = legendre(npg)
    
    for e in 1:ne
        monta_G_local!(G_local, e, c̄, EQoLG, P, W)
        
        for i in 1:2
            index = EQoLG[i, e]
            G_global[index] += G_local[i]
        end
    end
    
    return G_global[1:m]
end

monta_G_global (generic function with 1 method)

## Solver

In [18]:
function solve_system()
    
    LG = monta_LG(ne)
    m, EQ = monta_EQ(ne)
    EQoLG = EQ[LG]
    
    npg_K = 2
    npg_F = 5
    npg_G = 5

    K = monta_K_global(α, β, ne, npg_K, EQoLG) 
    
    N = ne
    τ = T/N
    t = 0:τ:T
    
    #caso inicial
    Cⁿ = zeros(m, 1)
    Cⁿ_1 = zeros(m, 1)
    
    #interpolando C0    
    Cⁿ_1 = u_0.(get_point.(1:m, m))
    
    #t1 predição
    c̄ = [Cⁿ_1; 0]
    G = monta_G_global(g, c̄, ne, m, npg_G, EQoLG)
    Fⁿ_meio = monta_F_global((x) -> f(x, (t[1]+t[2])/2, α, β), m, ne, npg_F, EQoLG )
    
    Cⁿ = K \ (τ * Fⁿ_meio + K * Cⁿ_1 - τ * G)

    #t1 correção

    c = 1/2 * (Cⁿ + Cⁿ_1)
    G = monta_G_global(g, [c; 0], ne, m, npg_G, EQoLG)

    Cⁿ = K \ (τ * Fⁿ_meio + K * Cⁿ_1 - τ * G)

    erros_iteracoes = []
    push!(erros_iteracoes, erro_L2_parabolico_2((x) -> u(x, t[1], α, β),[Cⁿ_1;0], ne))

    Cⁿ_2 = Cⁿ_1
    Cⁿ_1 = Cⁿ
    
    #display(ne)
    #display(EQoLG)
    #display(M)
    #display(K)
    #display(A)
    #display(B)
    
    for n in 2:N
        
        c = (3*Cⁿ_1 - Cⁿ_2)/2    
        G = monta_G_global(g, [c; 0], ne, m, npg_G, EQoLG)

        #display("Cn-1")
        #display(Cⁿ_1)
        #display("t_medio")
        #display((t[n]+t[n+1])/2)
        
        Fⁿ_meio = monta_F_global((x) -> f(x, (t[n]+t[n+1])/2, α, β), ne, EQoLG)
        
        #display("Fn_meio")
        #display(Fⁿ_meio)
        
        Cⁿ = K \ (τ * Fⁿ_meio + K * Cⁿ_1 - τ * G)
        
        #display("Cn")
        #display(Cⁿ)  
        
        uh = monta_uh(Cⁿ)
        Cⁿ_2 = Cⁿ_1
        Cⁿ_1 = Cⁿ
        
        #display("uh:")
        #display(uh)
        display(plot_exact_and_numerical(ne, u, t[n+1], α, β, uh, true))

        erro = erro_L2_parabolico_2((x) -> u(x, t[n+1], α, β), uh, ne)
        push!(erros_iteracoes, erro)
    end
    
    display("Erros L2")
    display(erros_iteracoes)
    display("Máximo erro L2")
    display(maximum(erros_iteracoes))
    return
end

solve_system (generic function with 1 method)

## Erro L2

In [17]:
function erro_L2(u, c̄, ne, npg)
    h = 1.0/ne
    
    P, W = legendre(npg)
    erro_quadrado = 0

    for e in 1:ne
        x_i_inf = get_point(e, ne)

        for (ξ, W) in zip(P, W)
            erro_quadrado += W * (u(map_ξ_to_x(ξ, x_i_inf, h)) - c̄[e]*ϕ₁(ξ) - c̄[e+1]*ϕ₂(ξ))^2 
        end
    end
    
    return sqrt(erro_quadrado * h/2)    
end

erro_L2 (generic function with 1 method)